In [11]:
#
# train_diabetes.py
#
#   MLflow model using ElasticNet (sklearn) and Plots ElasticNet Descent Paths
#
#   Uses the sklearn Diabetes dataset to predict diabetes progression using ElasticNet
#       The predicted "progression" column is a quantitative measure of disease progression one year after baseline
#       http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html
#   Combines the above with the Lasso Coordinate Descent Path Plot
#       http://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html
#       Original author: Alexandre Gramfort <alexandre.gramfort@inria.fr>; License: BSD 3 clause
#
#  Usage:
#    python train_diabetes.py 0.01 0.01
#    python train_diabetes.py 0.01 0.75
#    python train_diabetes.py 0.01 1.0
#

import sys
import warnings
from itertools import cycle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import ElasticNet, enet_path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Load Diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

# Create pandas DataFrame for sklearn ElasticNet linear_model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = diabetes.feature_names + ["progression"]
data = pd.DataFrame(d, columns=cols)


# Import mlflow
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature


# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
    train_x = train.drop(["progression"], axis=1)
    test_x = test.drop(["progression"], axis=1)
    train_y = train[["progression"]]
    test_y = test[["progression"]]

    #alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.05
    #l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.05
    mlflow.set_experiment("Diabetes Demo")
    alpha = 0.01
    l1_ratio = 0.50
    with mlflow.start_run() as run:
        # Run ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out ElasticNet model metrics
        print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
        print(f"  RMSE: {rmse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

        # Infer model signature
        predictions = lr.predict(train_x)
        signature = infer_signature(train_x, predictions)

        # Log mlflow attributes for mlflow UI
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(lr, "model", signature=signature,
                                 registered_model_name='diabmodel01'
                                 )

        # Compute paths
        eps = 5e-3  # the smaller it is the longer is the path

        print("Computing regularization path using the elastic net.")
        alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio)

        # Display results
        fig = plt.figure(1)
        ax = plt.gca()

        colors = cycle(["b", "r", "g", "c", "k"])
        neg_log_alphas_enet = -np.log10(alphas_enet)
        for coef_e, c in zip(coefs_enet, colors):
            l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle="--", c=c)

        plt.xlabel("-Log(alpha)")
        plt.ylabel("coefficients")
        title = "ElasticNet Path by alpha for l1_ratio = " + str(l1_ratio)
        plt.title(title)
        plt.axis("tight")

        # Save figures
        fig.savefig("ElasticNet-paths.png")

        # Close plot
        plt.close(fig)



Elasticnet model (alpha=0.010000, l1_ratio=0.500000):
  RMSE: 67.9835933653791
  MAE: 56.73850123875366
  R2: 0.3011911628749866
Computing regularization path using the elastic net.


Registered model 'diabmodel01' already exists. Creating a new version of this model...
Created version '3' of model 'diabmodel01'.
